## Prepare Dataset

In [13]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device count: ", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print("Device", i, "name: ", torch.cuda.get_device_name(i))
print("Device: ", device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Total Memory:    ', round(torch.cuda.get_device_properties(0).total_memory/1024**3,1), 'GB')
    print('Allocated Memory:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached Memory:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Device count:  1
Device 0 name:  NVIDIA GeForce RTX 4070 SUPER
Device:  cuda
NVIDIA GeForce RTX 4070 SUPER
Total Memory:     12.0 GB
Allocated Memory: 0.5 GB
Cached Memory:    0.8 GB


In [14]:
import pandas as pd
import json


# Dental dataset
#from datasets import load_dataset
#ds = load_dataset("Lines/Open-Domain-Oral-Disease-QA-Dataset")

path = "e:/git/cs224u/70-fixed-batch-inference.xlsx"

# Load dataset
df = pd.read_excel(path, sheet_name="review")
print('Original dataset length', len(df))

# Filter dataset
df = df.drop(df[df['reference_output_69'].str.find('PARSE_FAILED') != -1].index)
print('Filtered dataset length', len(df))

df['output'] = df.apply(lambda x: json.loads(x['reference_output_69']), axis=1)
df['input'] = df['raw_input'].copy()
print("Max input length", df['input'].str.len().max())

c = {}
for index, row in df.iterrows():
    x = row['output']
    for key, value in x.items():
        c[key] = c.get(key, 0) + int(len(value) > 0)

categories = sorted(list(c.keys()))

# Validation of the dataset
for index, row in df.iterrows():
    a=[0] * len(categories)
    try:
        x = json.loads(row['reference_output_69'])
        if type(x) is dict:
            # Check dental features for ex.
            if 'dental_features' in x and len(x['dental_features'])>0:
                a.append(1)
            else:
                a.append(0)
        else:
            print(x['answer'])
    except ValueError:  # includes simplejson.decoder.JSONDecodeError
        print('Decoding JSON has failed')

# print(df['output'].loc[df.index[2]])
for i in categories:
    print(c[i], '\t', i)
df['output_vector'] = df.apply(lambda x: [int(len(x['output'].get(i, [0])) > 0) for i in categories], axis=1)
df['output_vector']

Original dataset length 5492
Filtered dataset length 5451
Max input length 2078
2141 	 alignment
395 	 as_previous
1197 	 bite
181 	 crowding
1375 	 dental_features
472 	 finishing
485 	 leveling
272 	 midline
95 	 non_clinical_reason_for_new_order
1174 	 occlusion
1833 	 other_instructions
484 	 overcorrection_aligners
689 	 passive_aligners
1630 	 polite_expressions
55 	 request_for_clin_check
199 	 skip_active_treatment
1272 	 spaces
2648 	 teeth_movements
124 	 tracking
394 	 treatment_length


0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...
1       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...
3       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
4       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
5487    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5488    [1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...
5489    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...
5490    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5491    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: output_vector, Length: 5451, dtype: object

## Wrap the dataset into the class

In [15]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW


# Prepare the dataset
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        # Tokenize the text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            #return_attention_mask=True,
            return_attention_mask=False,
            return_tensors='pt'
        )

        # Take into account that the 
        return {
            'input_ids': encoding['input_ids'].flatten().to(device),
            #'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float, device=device)  # Float for multi-label
        }

# Prepare data for training
#X_train, X_val, y_train, y_val = train_test_split(df['input'], df['output_vector'], test_size=0.2)

In [16]:
from sklearn.model_selection import train_test_split

# Prepare data for training
X_train, X_val, y_train, y_val = train_test_split(df['input'].tolist(), df['output_vector'].tolist(), test_size=0.2, shuffle=False)


# Validate one of the examples
k = 0
print(X_val[0], y_val[0])
for i in range(len(categories)):
    print(categories[i], y_val[0][i])

[NumberingSystem]unknown

[FormInstructionsUpperArch:]please follow my clinical preferences. Thank you!

[FormInstructionsLowerArch:]please follow my clinical preferences. Thank you!

 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
alignment 0
as_previous 0
bite 0
crowding 0
dental_features 0
finishing 0
leveling 0
midline 0
non_clinical_reason_for_new_order 0
occlusion 0
other_instructions 1
overcorrection_aligners 0
passive_aligners 0
polite_expressions 1
request_for_clin_check 0
skip_active_treatment 0
spaces 0
teeth_movements 0
tracking 0
treatment_length 0


In [17]:
from tqdm import trange, tqdm

# Load the tokenizer and model
model_name = 'roberta-base'  # Change as needed

device_name = device.type + ":" + str(torch.cuda.current_device())

tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Max sentence", tokenizer.model_max_length)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(categories), device_map = device_name)

# Freeze all existing parameters
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the classification head
for param in model.classifier.parameters():
    param.requires_grad = True

# Print out which layers are trainable for verification
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Trainable layer: {name}")

print(model)

c:\Users\sgrebenkin\.conda\envs\cs224u\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



Max sentence 512


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable layer: classifier.dense.weight
Trainable layer: classifier.dense.bias
Trainable layer: classifier.out_proj.weight
Trainable layer: classifier.out_proj.bias
RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(

In [18]:
from sklearn.metrics import accuracy_score, f1_score
import torch.nn as nn
import numpy as np


name = 'roberta_multiclass_2_last_layers'

# Create datasets
train_dataset = CustomDataset(X_train, y_train, tokenizer)
val_dataset = CustomDataset(X_val, y_val, tokenizer)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

def binary_cross_entropy(t, p):
    t = np.float64(t)
    p = np.float64(p)
    # binary cross-entropy loss
    return -np.sum(t * np.log(p) + (1 - t) * np.log(1 - p))/len(t)


# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    predictions_f, true_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids']
            #attention_mask = batch['attention_mask']
            labels = batch['labels']

            #outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            outputs = model(input_ids=input_ids)
            preds = torch.sigmoid(outputs.logits).cpu().numpy()
            predictions_f.extend(preds)
            true_labels.extend(labels.cpu().numpy())

    predictions = [(p > 0.5).astype(int) for p in predictions_f]
    accuracy = accuracy_score(true_labels, predictions)
    f1score = f1_score(true_labels, predictions, average='macro')
    f1score_micro = f1_score(true_labels, predictions, average='micro')
    loss = binary_cross_entropy(true_labels, predictions_f)
    print(f'Validation Accuracy: {accuracy:.4f}, F1 Score: {f1score:.4f}, F1 Score micro: {f1score_micro:.4f}, Loss: {loss}')

    return (accuracy, f1score, f1score_micro, loss, predictions_f, true_labels)


# Training function
def train_model(model, train_loader, val_loader, epochs=10):
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

    losses = []
    accuracies = []
    f1scores = []
    f1scores_micro = []
    val_losses = []
    predictions = []
    true_labels = []

    current_f1score_micro = 0

    for epoch in range(epochs):
        model.train() # set dropout to train mode
        for param in model.parameters():
            param.requires_grad = False

        # Last layer unblock
        for param in model.roberta.encoder.layer[-1].parameters():
            param.requires_grad = True
        
        # Unfreeze the classification head
        for param in model.classifier.parameters():
            param.requires_grad = True

        for batch in tqdm(train_loader):
            optimizer.zero_grad()
            input_ids = batch['input_ids']
            #attention_mask = batch['attention_mask']
            labels = batch['labels']
            
            #outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{epochs} completed. Train loss: {loss.item()}')
        losses.append(loss.item())
    
        (accuracy, f1score, f1score_micro, val_loss, predictions_f, true_labels) = evaluate_model(model, val_loader)
        accuracies.append(accuracy)
        f1scores.append(f1score)
        f1scores_micro.append(f1score_micro)
        val_losses.append(val_loss)

        if f1score_micro > current_f1score_micro:
            current_f1score_micro = f1score_micro
            model.save_pretrained(name)
            tokenizer.save_pretrained(name)

    return (losses, accuracies, f1scores, f1scores_micro, val_losses, predictions_f, true_labels)

In [19]:
%env CUDA_LAUNCH_BLOCKING=0

import os
print(os.environ["CUDA_LAUNCH_BLOCKING"])

# Train the model
(losses, accuracies, f1scores, f1scores_micro, val_losses, predictions_f, true_labels) = train_model(model, train_loader, val_loader, epochs=50)

env: CUDA_LAUNCH_BLOCKING=0
0


100%|██████████| 2180/2180 [00:51<00:00, 42.54it/s]


Epoch 1/50 completed. Train loss: 0.35479891300201416
Validation Accuracy: 0.0348, F1 Score: 0.0798, F1 Score micro: 0.2981, Loss: 7.211238115751399


100%|██████████| 2180/2180 [00:50<00:00, 43.04it/s]


Epoch 2/50 completed. Train loss: 0.4234897792339325
Validation Accuracy: 0.0165, F1 Score: 0.0769, F1 Score micro: 0.2369, Loss: 7.092878521547497


...

Epoch 48/50 completed. Train loss: 0.061468061059713364
Validation Accuracy: 0.2786, F1 Score: 0.6724, F1 Score micro: 0.7500, Loss: 3.7866736320672576


100%|██████████| 2180/2180 [00:51<00:00, 42.73it/s]


Epoch 49/50 completed. Train loss: 0.12640778720378876
Validation Accuracy: 0.2961, F1 Score: 0.6679, F1 Score micro: 0.7594, Loss: 3.6576538289847855


100%|██████████| 2180/2180 [00:50<00:00, 42.82it/s]


Epoch 50/50 completed. Train loss: 0.14332376420497894
Validation Accuracy: 0.2878, F1 Score: 0.6704, F1 Score micro: 0.7589, Loss: 3.6893000923854316


In [20]:
# Save the model
#model.save_pretrained(name)
#tokenizer.save_pretrained(name)

In [25]:
import plotly.express as px

df_loss = pd.DataFrame(losses, columns=['loss'])
df_loss["epoch"] = range(0, len(df_loss))

fig = px.line(df_loss, x="epoch", y="loss")
fig.show()

In [26]:
import pandas as pd

df_metrics = pd.DataFrame({'f1score': f1scores, 'f1score-micro': f1scores_micro,'val_losses': val_losses, 'val_accuracies': accuracies})
df_metrics["epoch"] = range(0, len(df_metrics))

fig = px.line(df_metrics, x="epoch", y="f1score")
fig.show()

In [27]:
fig = px.line(df_metrics, x="epoch", y="val_losses")
fig.show()

In [28]:
# (losses, accuracies, f1scores, f1scores_micro, val_losses, predictions_f, true_labels)
with open(name + "/losses.txt", "w") as f:
    for i in losses:
        f.write(str(i)+"\n")
with open(name + "/accuracies.txt", "w") as f:
    for i in accuracies:
        f.write(str(i)+"\n")
with open(name + "/f1scores.txt", "w") as f:
    for i in f1scores:
        f.write(str(i)+"\n")
with open(name + "/f1scores_micro.txt", "w") as f:
    for i in f1scores_micro:
        f.write(str(i)+"\n")
with open(name + "/val_losses.txt", "w") as f:
    for i in val_losses:
        f.write(str(i)+"\n")